In [7]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectPercentile, SelectKBest, RFECV, f_regression 
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import  SVR

X_test = pd.read_csv("X_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

columns = X_train.columns[1:]

# convert to np arrays
X = X_train.iloc[:,1:].to_numpy()
y = y_train.iloc[:,1:].to_numpy().reshape(-1)
X_test = X_test.iloc[:,1:].to_numpy()


x_stack = np.vstack((X,X_test))
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=15)
x_stack = imputer.fit_transform(x_stack)
X = x_stack[:1212]
X_test = x_stack[1212:]

outliers = IsolationForest(n_estimators=1000, random_state=0).fit_predict(X)
#behaviour='new', max_samples=100, random_state = 1, contamination='auto'
X = X[np.where(outliers == 1)[0]].copy()
y = y[np.where(outliers == 1)[0]].copy()

selector = VarianceThreshold()
selector.fit(X)
to_delete = np.where(selector.variances_ == 0)[0]
X = np.delete(X, to_delete, axis=1)
X_test = np.delete(X_test, to_delete, axis=1)

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel, Matern, ExpSineSquared, ConstantKernel as C

kernel = Matern(length_scale=100.0, nu=10)

est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=100, random_state=0,loss='squared_error')
r1 = SVR(C=37, gamma=0.0219)
r2 = RandomForestRegressor(n_estimators=1000, random_state=1)
regr = make_pipeline(RobustScaler(), SelectKBest(f_regression, k=94), SVR(C=37, gamma=0.0219))#VotingRegressor([("est", est), ("r1", r1), ("r2", r2)]))
cv_results = cross_validate(regr, X, y, scoring=('r2'))
print(cv_results['test_score'].mean())

0.6081467485265141


In [8]:
SVC(class_weight='balanced')

SVC(class_weight='balanced')

In [9]:
X.shape

(1209, 828)

In [11]:
regr.fit(X, y)
y_pred = regr.predict(X_test)
X_test = pd.read_csv("X_test.csv")
df_ids = pd.DataFrame(X_test['id'])
df_predictions = df_ids.join(pd.DataFrame(y_pred, columns=['y']))
df_predictions.to_csv('model_results/svr_y_test.csv', index=False)

In [12]:
y_train_pred = regr.predict(X)
X_train = pd.read_csv("X_train.csv")
df_ids = pd.DataFrame(X_train['id'])
df_predictions = df_ids.join(pd.DataFrame(y_train_pred, columns=['y']))
df_predictions.to_csv('model_results/svr_y_train.csv', index=False)

In [15]:
y_train_pred

array([71.1001745 , 72.90013536, 66.10019421, ..., 81.89975404,
       76.09983803, 80.89981725])

In [16]:
y

array([71., 73., 66., ..., 82., 76., 81.])

In [17]:
df_predictions = df_ids.join(pd.DataFrame(y, columns=['y']))
df_predictions.to_csv('model_results/y_train.csv', index=False)